In [1]:
import torch.utils.data as data
from chess_dataset import ChessDataset

dataset = ChessDataset("tests/test.engine.csv")
data_loader = data.DataLoader(dataset, batch_size=4, shuffle=True)

In [2]:
import torch
import torch. nn as nn
from models import ChessEngine

c = ChessEngine(23, 128, 11)
if torch.cuda.device_count() > 1:
    print('Let\'s use', torch.cuda.device_count(), 'GPUs!')
    c = nn.DataParallel(c)
if torch.cuda.is_available():
    print('Cuda!')
    c.cuda()

Let's use 2 GPUs!
Cuda!


In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(c.parameters(), lr=0.001, momentum=0.9, nesterov=True)

In [4]:
%%timeit -n 1 -r 10

from torch.autograd import Variable

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = c(inputs)
        outputs = outputs.view(outputs.shape[0], -1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
print('Training done')

Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
1.7 s ± 851 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
